In [3]:
import numpy as np 
import pandas as pd 
'''
import os
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
%matplotlib notebook
'''
#LSTM은 오래 이전의 값을 보존한다. 이때, cell gate는 학습을 안하고 보존하는값 forget 학습시켜 얼마나 잊을지 
# https://wikidocs.net/45101 : RNN을 이용한 텍스트 생성(Text Generation using RNN) 
# https://www.kaggle.com/somang1418/youtube-video-title-generator-by-lstm-eda : Kaggle사이트

"\nimport os\nimport matplotlib.pyplot as plt\n%matplotlib inline\nplt.style.use('ggplot')\nimport seaborn as sns\n%matplotlib notebook\n"

In [4]:
from tqdm import tqdm 
import string

In [5]:
# spacy based imports 
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English 

In [6]:
# keras module for building LSTM  
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer 
from keras.callbacks import EarlyStopping 
from keras.models import Sequential 
import keras.utils as ku  

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [7]:
# spaCy based imports 
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS 
from spacy.lang.en import English  

In [8]:
us_youTube = pd.read_csv("USvideos.csv") 
#ca_youTube = pd.read_csv("CAvideos.csv")
#gb_youTube = pd.read_csv("GBvideos.csv")

#파일 읽어온후, 병합 
#country_youTubeSet = pd.concat([us_youTube,ca_youTube]) 
#country_youTubeSet = us_youTube

In [9]:
#country_youTubeSet = pd.concat([us_youTube,ca_youTube,gb_youTube]) 
country_youTubeSet = us_youTube
#remove duplicate  
country_youTubeSet= country_youTubeSet.drop_duplicates(['video_id'], keep='first') 
#dataframe에서 중복값 발생,first occurance만 제외하고 drop시킴
 
country_youTubeSet.video_id.value_counts()[:10] 
 

sRbhD2unwBc    1
dHCURII_974    1
hlb3gGNBHFE    1
c0mbyMWd0Q8    1
TaVvm0r_gho    1
XPVxL-YUOwk    1
9mwM1VB9qs0    1
-xt_j2aIAkg    1
ZWMp9BpWODk    1
OdpHOc7vqHA    1
Name: video_id, dtype: int64

In [10]:
print(type(country_youTubeSet.views[0]))
country_youTubeSet.describe()

<class 'numpy.int64'>


,category_id,views,likes,dislikes,comment_count
count,6351.000000,6.351000e+03,6.351000e+03,6351.000000,6351.000000
mean,20.444182,7.582096e+05,3.449357e+04,1427.222012,4501.707290
std,7.213046,1.928993e+06,1.162439e+05,12078.814524,21460.223331
min,1.000000,5.490000e+02,0.000000e+00,0.000000,0.000000
25%,17.000000,8.351100e+04,1.908000e+03,72.000000,261.000000
50%,24.000000,2.709020e+05,7.987000e+03,242.000000,921.000000
75%,25.000000,7.512665e+05,2.516300e+04,763.000000,2845.000000
max,43.000000,4.843165e+07,3.880071e+06,629120.000000,733373.000000


In [11]:
#need to be decoded  country_youTubeSet.category_id.head() 
 
Q1 = country_youTubeSet.views.quantile(0.25)
Q3 = country_youTubeSet.views.quantile(0.75) 
# 분위수 조절 
print(Q3,'\n',Q1)
IQR = Q3-Q1 

751266.5 
 83511.0


In [12]:
# 기초 통계: IQR(중간에 50%의 데이터들이 흩어진 정도) 
# 데이터 집합에서 경계값을 포함한 4분위수
# Q1: 제 1사분위수 / Q2: 중앙값 / Q3: 제 3사분위수/ Q4: 최대값   
# 중앙값 범위를 IQR이라 할때, 아래 범위를 Q1, 위의 범위를 Q3라고 함
# 사분위수 범위(IQR, Interquartile range) = Q3 - Q1 
#안울타리(Inner fence) = Q1 - 1.5 * IQR 과 Q3 + 1.5 * IQR #바깥울타리(Outer fence) = Q1 - 3 * IQR 과 Q3 + 3 * IQR 
#출처: https://m.blog.naver.com/PostView.nhn?blogId=cooroom&logNo=100206895658&proxyReferer=https %3A%2F%2Fwww.google.com%2F 

In [13]:
popular_videos = country_youTubeSet.loc[country_youTubeSet.views>Q3+1.5*IQR] 
# 실제 사용되지 않는 셀이지만, 다음 처럼 인기있는 비디오를 추출할 수 있음 
# 'Q3+1.5*IQR'의 의미  
# 안울타리(Inner fence)로서 울라티를 벗어난 값을 이상점으로 판단 

country_youTubeSet['popular']=0 
country_youTubeSet.loc[country_youTubeSet.views > (Q3 + 1.5 * IQR),'popular']=1 
country_youTubeSet['popular'].value_counts() 
print('popular video의 비율:',len(country_youTubeSet[country_youTubeSet['popular']==1])/
      len(country_youTubeSet[country_youTubeSet['popular']==0])*100,' %') 
# us_youTube dataframe에 popular라는 속성을 만들고 0값으로 초기화 
# 이후, 이상점(view수가 많은)데이터의 경우, popular속성에 1을 삽입 
# 결과 적으로  
 
spacy_stopwords = STOP_WORDS 
print('Number of stop words:%d' % len(spacy_stopwords))
print('First ten stop words: %s' % list(spacy_stopwords)[:10])
# 사전에 정의된 stop words  
# https://medium.com/@makcedward/nlp-pipeline-stop-words-part-5-d6770df8a936 

popular video의 비율: 10.529063696484512  %
Number of stop words:326
First ten stop words: ['perhaps', 'where', "n't", 'themselves', 'me', 'seems', 'always', 're', 'as', 'us']


In [14]:
punctuations = string.punctuation 
#문장 부호(!@#<>?<) 선별 for 단어만을 뽑아내기 위해, 
#https://www.geeksforgeeks.org/string-punctuation-in-python/
#LSTM으로 학습시킬때, 주-목-보 => 보어가 주목에 영향 줄 수 있음

stopwords = list(STOP_WORDS) 
parser = English()  
#영어 불용어 리스트를 전환  
# I, my, me, over, 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는데는 거의 기여하는 바가 없습니다. 

In [15]:
new_stopwords= ['trailer','Official','vs','Video' ]
stopwords.extend(new_stopwords) #append로 넣을 경우, 리스트 형태를 포함해서 들어간다. extend는 값만 들어감
stopwords

['perhaps',
 'where',
 "n't",
 'themselves',
 'me',
 'seems',
 'always',
 're',
 'as',
 'us',
 'whereby',
 'hence',
 'put',
 'a',
 'either',
 'below',
 'without',
 "'s",
 'unless',
 'but',
 'eleven',
 'would',
 'beside',
 'anyway',
 'anywhere',
 'over',
 'three',
 'really',
 'becomes',
 'mine',
 'do',
 'about',
 'made',
 'yourselves',
 'thru',
 'done',
 'hereafter',
 'last',
 'or',
 'hers',
 'my',
 'even',
 'any',
 'we',
 '’d',
 'beyond',
 'anyone',
 '’re',
 'amongst',
 'someone',
 'front',
 'moreover',
 'due',
 'has',
 'yet',
 'seem',
 'into',
 'to',
 'might',
 'then',
 'is',
 'down',
 'for',
 'used',
 'therein',
 'several',
 'see',
 'across',
 'third',
 'be',
 'until',
 'become',
 'such',
 'than',
 'one',
 'himself',
 'whereafter',
 'whereupon',
 'been',
 'part',
 'anything',
 'throughout',
 'first',
 'twelve',
 '’s',
 'were',
 'becoming',
 'noone',
 'few',
 'somehow',
 'will',
 'at',
 '‘s',
 'around',
 'bottom',
 'out',
 'therefore',
 'twenty',
 'their',
 '’ll',
 'something',
 'two'

In [16]:
def spacy_tokenizer(sentence):  
    
    mytokens = parser(sentence) 
    #sentence의 type: str -> mytokens의 type: spacy.doc.Doc (parsing과정을 통해, 데이터의 type변경) 
    #print(mytokens[word_lemma for word in mytokens])   
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ] 
    #lower() 문자열 내 모든 대문자를 모두 소문자로 변환, strip 문자 양쪽 한칸 이상의 연속된 공백지움
    #word.lemma는 단어의 원형 형태 ex) is -> be, growing ->grow 
    #pron -> pronoun 대명사, => 대명사가 아닐경우, 원형+소문자+공백제거 / 대명사 일 경우, 소문자
    print("1:",mytokens)          
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]   
    print("2:",mytokens) #stopwords(설정된 금지단어) 와 punctuations(특수 부호)조건 모두에 포함 되지 않은 단어만 추출   
    mytokens = " ".join([i for i in mytokens]) 
    # [,,,]로 구분되어있던 token을 다시 문장으로 합침 -> join ""로 했기 때문에 str으로 변경     
    print("3:",(mytokens)) 
    return mytokens 

In [17]:
tqdm.pandas() #tqdm.pands 초기화 initialze 
normal = country_youTubeSet["title"][country_youTubeSet["popular"] == 0].progress_apply(spacy_tokenizer)
popular = country_youTubeSet["title"][country_youTubeSet["popular"] == 1].progress_apply(spacy_tokenizer) 
#progress_apply: tqdm을 사용하면 row개수와 row하나를 처리하는데 걸리는 수행시간을 알려줌 

  0%|                                                                                         | 0/5746 [00:00<?, ?it/s]

1: ['we', 'want', 'to', 'talk', 'about', 'our', 'marriage']
2: ['want', 'talk', 'marriage']
3: want talk marriage
1: ['nickelback', 'lyrics', ':', 'real', 'or', 'fake', '?']
2: ['nickelback', 'lyrics', 'real', 'fake']
3: nickelback lyrics real fake
1: ['2', 'weeks', 'with', 'iphone', 'x']
2: ['2', 'weeks', 'iphone', 'x']
3: 2 weeks iphone x
1: ['5', 'ice', 'cream', 'gadgets', 'put', 'to', 'the', 'test']
2: ['5', 'ice', 'cream', 'gadgets', 'test']
3: 5 ice cream gadgets test
1: ['the', 'greatest', 'showman', '|', 'official', 'trailer', '2', '[', 'hd', ']', '|', '20th', 'century', 'fox']
2: ['greatest', 'showman', 'official', '2', 'hd', '20th', 'century', 'fox']
3: greatest showman official 2 hd 20th century fox
1: ['why', 'the', 'rise', 'of', 'the', 'robots', 'will', 'not', 'mean', 'the', 'end', 'of', 'work']
2: ['rise', 'robots', 'mean', 'end', 'work']
3: rise robots mean end work
1: ['dion', 'lewis', "'", '103-yd', 'kick', 'return', 'td', 'vs.', 'denver', '!', '|', "can't", '-', 'miss

  0%|▏                                                                              | 13/5746 [00:00<00:47, 120.69it/s]

1: ['one', 'change', 'that', 'would', 'make', 'pacific', 'rim', 'a', 'classic']
2: ['change', 'pacific', 'rim', 'classic']
3: change pacific rim classic
1: ['how', 'does', 'your', 'body', 'know', 'you', 'be', 'full', '?', '-', 'hilary', 'coller']
2: ['body', 'know', 'hilary', 'coller']
3: body know hilary coller
1: ['homemade', 'electric', 'airplane']
2: ['homemade', 'electric', 'airplane']
3: homemade electric airplane
1: ['founding', 'an', 'inbreeding', '-', 'free', 'space', 'colony']
2: ['founding', 'inbreeding', 'free', 'space', 'colony']
3: founding inbreeding free space colony
1: ['how', 'can', 'you', 'control', 'your', 'dreams', '?']
2: ['control', 'dreams']
3: control dreams
1: ['the', 'making', 'of', 'hela', "'s", 'headdress', 'from', 'thor', ':', 'ragnarok']
2: ['making', 'hela', 'headdress', 'thor', 'ragnarok']
3: making hela headdress thor ragnarok
1: ['is', 'it', 'dangerous', 'to', 'talk', 'to', 'a', 'camera', 'while', 'driving', '?']
2: ['dangerous', 'talk', 'camera', 'dr

  0%|▎                                                                              | 25/5746 [00:00<00:48, 119.15it/s]

...

 98%|█████████████████████████████████████████████████████████████████████████████▍ | 593/605 [00:06<00:00, 102.02it/s]

1: ['shawn', 'mendes', '-', 'nervous']
2: ['shawn', 'mendes', 'nervous']
3: shawn mendes nervous
1: ['halo', 'infinite', '-', 'e3', '2018', '-', 'announcement', 'trailer']
2: ['halo', 'infinite', 'e3', '2018', 'announcement']
3: halo infinite e3 2018 announcement
1: ['big', 'shaq', '-', 'man', "don't", 'dance', '(', 'official', 'music', 'video', ')']
2: ['big', 'shaq', 'man', "don't", 'dance', 'official', 'music', 'video']
3: big shaq man don't dance official music video
1: ['yale', 'graduation', 'speaker', 'breaks', 'up', 'with', 'boyfriend', 'during', 'speech', '|', 'rebecca', 'shaw', 'and', 'ben', 'kronengold']
2: ['yale', 'graduation', 'speaker', 'breaks', 'boyfriend', 'speech', 'rebecca', 'shaw', 'ben', 'kronengold']
3: yale graduation speaker breaks boyfriend speech rebecca shaw ben kronengold
1: ['how', 'black', 'panther', 'should', 'have', 'ended']
2: ['black', 'panther', 'ended']
3: black panther ended
1: ['nintendo', '@', 'e3', '2018', ':', 'day', '1']
2: ['nintendo', 'e3', '

100%|██████████████████████████████████████████████████████████████████████████████▊| 604/605 [00:06<00:00, 100.10it/s]

1: ['the', 'nun', '-', 'official', 'teaser', 'trailer', '[', 'hd', ']']
2: ['nun', 'official', 'teaser', 'hd']
3: nun official teaser hd
1: ['film', 'theory', ':', 'how', 'to', 'save', 'jurassic', 'park', '(', 'jurassic', 'world', ')']
2: ['film', 'theory', 'save', 'jurassic', 'park', 'jurassic', 'world']
3: film theory save jurassic park jurassic world


100%|████████████████████████████████████████████████████████████████████████████████| 605/605 [00:06<00:00, 96.64it/s]


In [21]:
''' 
 apply 구문 해석 
 현재 us_youTube["title"][us_youTube["popular"] == 0] 은 series이며, index값과 value값을 동시에 가진다. 
 이때, parser를 바로 해주면, index값에도 동시에 적용되기 때문에, 오류가 발생한다. 
 따라서, value값만을 함수에 적용할 필요가 있다. apply함수를 series1.apply(x)로 적용할 경우, 
 는 series1의 value 값을 가져온다 
 사실, serires1은 youTube데이터(dataframe)에서 속성을 title하라고 지정하였기 때문에,
 serires로 타입이 변환된다.   
 아래 예시를 보면, serires1에서 str값에 해당하는 index를 제외한 반환 값만이 적용된다.   
 
 def test(x):     
     print(type(x),x) 
 us_youTube["title"][us_youTube["popular"] == 0].apply(test) 
 '''

' \n apply 구문 해석 \n 현재 us_youTube["title"][us_youTube["popular"] == 0] 은 series이며, index값과 value값을 동시에 가진다. \n 이때, parser를 바로 해주면, index값에도 동시에 적용되기 때문에, 오류가 발생한다. \n 따라서, value값만을 함수에 적용할 필요가 있다. apply함수를 series1.apply(x)로 적용할 경우, \n 는 series1의 value 값을 가져온다 \n 사실, serires1은 youTube데이터(dataframe)에서 속성을 title하라고 지정하였기 때문에,\n serires로 타입이 변환된다.   \n 아래 예시를 보면, serires1에서 str값에 해당하는 index를 제외한 반환 값만이 적용된다.   \n \n def test(x):     \n     print(type(x),x) \n us_youTube["title"][us_youTube["popular"] == 0].apply(test) \n '

In [22]:
# convert data to sequence of tokens  
tokenizer = Tokenizer() 
#tokenize 함수는 어휘 스캐너로서 하나의 인자 readline을 요구  
def get_sequence_of_tokens(corpus):     
    ## tokenization          
    #만약 다음 함수를 쓰면, tokenizer = Tokenizer(num_words=1000)     
    #=> 가장 빈도가 높은 1,000개의 단어만 선택하도록 Tokenizer 객체를 만든다.         
    tokenizer.fit_on_texts(corpus)     
    # 단어 인덱스를 구축합니다. 각각의 단어마다 고유의 index가 할당됨    
    # => tokenizer.fit_on_texts(samples)    
    # 문자열을 정수 인덱스의 리스트로 변환합니다.  
    # => sequences = tokenizer.texts_to_sequences(samples)    
    total_words = len(tokenizer.word_index) + 1      
    #전체 단어의 수(인덱스가 0부터이기 때문에, +1을 할경우, 전체 수를 나타낸다.) 
     ## convert data to sequence of tokens     
    input_sequences = []          
    for line in corpus: # n번 반복
        token_list = tokenizer.texts_to_sequences([line])[0] # 현재 단어에 대한 정수 인코딩 
        for i in range(1, len(token_list)):          
            n_gram_sequence = token_list[:i+1]              
            input_sequences.append(n_gram_sequence)    
    return input_sequences, total_words 
 
inp_sequences, total_words = get_sequence_of_tokens(popular) 
print(inp_sequences[:10]) 
for i in inp_sequences[:10]:    
    print(i)     
    print(max(i)) 
    #print("최대 길이:",max(inp_sequences[:15])) 

[[42, 607], [42, 607, 32], [42, 607, 32, 202], [42, 607, 32, 202, 33], [42, 607, 32, 202, 33, 143], [42, 607, 32, 202, 33, 143, 96], [608, 609], [608, 609, 339], [608, 609, 339, 340], [608, 609, 339, 340, 144]]
[42, 607]
607
[42, 607, 32]
607
[42, 607, 32, 202]
607
[42, 607, 32, 202, 33]
607
[42, 607, 32, 202, 33, 143]
607
[42, 607, 32, 202, 33, 143, 96]
607
[608, 609]
609
[608, 609, 339]
609
[608, 609, 339, 340]
609
[608, 609, 339, 340, 144]
609


In [54]:
def generate_padded_sequences(input_sequences):  #데이터에 대한 패딩
    max_sequence_len = max([len(x) for x in input_sequences])    
    #print(type(input_sequences))     
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))     
    #자연어 처리에서는 패딩(padding) 작업이라고 하는데, 보통 숫자 0을 넣어서 길이가 다른 샘플들의 길이를 맞춰줍니다    
    #https://wikidocs.net/32105   
    #print(input_sequences[:,:-1][0])
    #예측값: 단어 시퀀스(라벨 다음 단어들)    
    #print(input_sequences[:,-1])#입력값: 현재 라벨    
    #print(input_sequences)    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]   
    print(input_sequences)
    print("$$$")
    print(input_sequences[:,:-1])
    print(type(input_sequences))
    print(input_sequences[:,-1])
    '''
    X = sequences[:,:-1]
    y = sequences[:,-1]
    # 리스트의 마지막 값을 제외하고 저장한 것은 X
    # 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.
    '''
    
    label = ku.to_categorical(label, num_classes=total_words)    
    #print(label[:,:][50])# 라벨링     
    #https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical  
    return predictors, label, max_sequence_len 
 

In [55]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences) 

print("predictors:",predictors[0]) 
print("label:",label[0]) 

[[  0   0   0 ...   0  42 607]
 [  0   0   0 ...  42 607  32]
 [  0   0   0 ... 607  32 202]
 ...
 [  0   0   0 ... 385  61 518]
 [  0   0   0 ...  61 518  61]
 [  0   0   0 ... 518  61  11]]
$$$
[[  0   0   0 ...   0   0  42]
 [  0   0   0 ...   0  42 607]
 [  0   0   0 ...  42 607  32]
 ...
 [  0   0   0 ...  31 385  61]
 [  0   0   0 ... 385  61 518]
 [  0   0   0 ...  61 518  61]]
<class 'numpy.ndarray'>
[607  32 202 ... 518  61  11]
predictors: [ 0  0  0  0  0  0  0  0  0  0  0  0  0 42]
label: [0. 0. 0. ... 0. 0. 0.]


In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    #y데이터를 분리하였기 때문에, x데이터의 길이는 기존 데이터의 길이 -1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    #embedding 단어를 벡터화하고, 이 결과값을 LSTM에 집어 넣어줌 
    #=> 단어를 의미론적 기하공간에 매핑할 수 있도록 벡터화 시킴 
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100)) #100으 ㅣ은닉 상태 크기를 가진 LSTM
    model.add(Dropout(0.1))
    #여러개 매핑을 넣어서, 서로 곱하고 더하여, output h(t)를 고려한다.
    # => short term memory는 h(t)와h(t-1)만들 고려해도 되지만,      
    # => long term memory인 LSTM의 경우, cell gate의 복잡성을 더해 4개의 매핑을 만들어 output h(t)를 만들어 낸다.
     # non-recurrent한 연결만 dropout을 적용하면 과거 중요 정보를 희생하지 않아도 regularization 
    # Add Output Layer
    
    model.add(Dense(total_words, activation='softmax'))
    #[ ~,~ , ~,~] 확률로 output이 제공되는데, softmax를 사용하면, 모든 값의 합이 1이어야한다.
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    # 범주형 변수를 예측할떄,2개: binary_crossentropy / 3개 이상: categorical_crossentropy     
    # loss 비용함수 설정, optimizer 인수로 최적화 알고리즘 설정 
    return model

model = create_model(max_sequence_len, total_words)
model.summary()
model.fit(predictors, label, epochs=300, verbose=2)

In [94]:
#신경망 구현 순서 
'''
Keras 를 사용하면 다음과 같은 순서로 신경망을 구성할 수 있다.
Sequential 모형 클래스 객체 생성
add 메서드로 레이어 추가. 
입력단부터 순차적으로 추가한다.
레이어는 출력 뉴런 갯수를 첫번째 인수로 받는다.
최초의 레이어는 input_dim 인수로 입력 크기를 설정해야 한다.
activation 인수로 활성화함수 설정
compile 메서드로 모형 완성. 
loss인수로 비용함수 설정
optimizer 인수로 최적화 알고리즘 설정
metrics 인수로 트레이닝 단계에서 기록할 성능 기준 설정
fit 메서드로 트레이닝
nb_epoch 로 에포크(epoch) 횟수 설정
batch_size 로 배치크기(batch size) 설정
verbose는 학습 중 출력되는 문구를 설정하는 것으로, 주피터노트북(Jupyter Notebook)을 사용할 때는 verbose=2로 설정하여 진행 막대(progress bar)가 나오지 않도록 설정한다.

출처: https://datascienceschool.net/view-notebook/51e147088d474fe1bf32e394394eaea7/
'''

'\nKeras 를 사용하면 다음과 같은 순서로 신경망을 구성할 수 있다.\nSequential 모형 클래스 객체 생성\nadd 메서드로 레이어 추가. \n입력단부터 순차적으로 추가한다.\n레이어는 출력 뉴런 갯수를 첫번째 인수로 받는다.\n최초의 레이어는 input_dim 인수로 입력 크기를 설정해야 한다.\nactivation 인수로 활성화함수 설정\ncompile 메서드로 모형 완성. \nloss인수로 비용함수 설정\noptimizer 인수로 최적화 알고리즘 설정\nmetrics 인수로 트레이닝 단계에서 기록할 성능 기준 설정\nfit 메서드로 트레이닝\nnb_epoch 로 에포크(epoch) 횟수 설정\nbatch_size 로 배치크기(batch size) 설정\nverbose는 학습 중 출력되는 문구를 설정하는 것으로, 주피터노트북(Jupyter Notebook)을 사용할 때는 verbose=2로 설정하여 진행 막대(progress bar)가 나오지 않도록 설정한다.\n\n출처: https://datascienceschool.net/view-notebook/51e147088d474fe1bf32e394394eaea7/\n'

In [98]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):  #for _  표시는 단순 반복의미 _나 i나 같은 의미 
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        #입력한 X(token_list,현재단어)에 대해서 Y를 예측하고 prediced에 저장 
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted: #만약 예측한 단어의 인덱스와 동일한 단어가 있다면,
                output_word = word 
                #해당 단어word가 예측단어이므로, outword에 넣고 중단 후,
                #다음 단어로 
                break
        seed_text += " "+output_word
    return seed_text.title()

In [99]:
print (generate_text("Drake", 5, model, max_sequence_len))
print (generate_text("Kendrick", 5, model, max_sequence_len))
print (generate_text("BTS", 5, model, max_sequence_len))
print (generate_text("karaoke ", 5, model, max_sequence_len))
print (generate_text("samsung", 5, model, max_sequence_len))
print (generate_text("taylor", 5, model, max_sequence_len))

Drake 5 Girls Ft Cardi B
Kendrick Lamar U2 Dave Chappelle Performance
Bts 방탄소년단 Fake Love Official Mv
Karaoke  Mama Hong Kong Bts Bts
Samsung Galaxy S9 Worth Hype Nail
Taylor Swift Delicate Pt 2 Bts
